In [1]:
%load_ext autoreload
%autoreload 2

main_dir = '/Users/simon/bsose_monthly/'
salt = main_dir + 'bsose_i106_2008to2012_monthly_Salt.nc'
theta = main_dir + 'bsose_i106_2008to2012_monthly_Theta.nc'

In [2]:
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import pyxpcm
from pyxpcm.models import pcm


def pcm_fit_and_post(time_i=42, K=4, maxvar=2):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(0., -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=time_i)
    theta_nc = xr.open_dataset(theta).isel(time=time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > 
                           max_depth).drop(['iter', 'Depth', 
                                            'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
        
    lons_new = np.linspace(both_nc.XC.min(), both_nc.XC.max(), 60*4)
    lats_new = np.linspace(both_nc.YC.min(), both_nc.YC.max(), 60)
    # ds = both_nc # .copy(deep=True)
    ds = both_nc.interp(coords={'YC': lats_new, 'XC': lons_new})#, method='cubic')
        
    m = pcm(K=K, features=features_pcm, maxvar=maxvar, 
            timeit=True, timeit_verb=1)
    m.fit(ds, features=features, dim='Z') #, inplace=True)
    m.predict(ds, features=features, dim='Z', inplace=True)
    m.predict_proba(ds, features=features, dim='Z', inplace=True)
    m.find_i_metric(ds, inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        del ds.PCM_RANK.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    return ds, m

/Users/simon/pyxpcm/pyxpcm/plot.py:45: UserWarning: pyXpcm requires seaborn installed for full plotting functionality
  warnings.warn("pyXpcm requires seaborn installed for full plotting functionality")


In [3]:
ds, m = pcm_fit_and_post(K=5)
ds

  fit.1-preprocess.1-mask: 13 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  fit.1-preprocess.2-feature_THETA.1-ravel: 17 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02


[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict_proba.1-preprocess.2-feature_SALT.2-interp: 20 ms
  predict_proba.1-preprocess.2-feature_SALT.3-scale_fit: 0 ms
  predict_proba.1-preprocess.2-feature_SALT.4-scale_transform: 33 ms
  predict_proba.1-preprocess.2-feature_SALT.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_SALT.6-reduce_transform: 7 ms
  predict_proba.1-preprocess.2-feature_SALT.total: 80 ms
  predict_proba.1-preprocess: 80 ms
  

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pcm_class: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pcm_class
Data variables:
    SALT        (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCM_LABELS  (YC, XC) float64 nan nan nan nan nan nan ... 4.0 4.0 4.0 4.0 4.0
    PCM_POST    (pcm_class, YC, XC) float64 nan nan nan nan ... 1.0 1.0 1.0 1.0
    PCM_RANK    (pcm_class, YC, XC) float64 nan nan nan nan ... 1.0 1.0 1.0 1.0

In [4]:
ds.THETA

<xarray.DataArray 'THETA' (Z: 52, YC: 60, XC: 240)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.91224107, 16.43401331, 16.42829744, ..., 16.73668129,
         16.96183119, 17.00106982],
        [17.10594605, 16.58357543,         nan, ..., 17.78552694,
         17.77893846, 17.15627046],
        [17.39700127, 17.41902259, 17.57648229, ..., 18.07164888,
         18.36557512, 17.36038017]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.91124901, 16.43339274, 16.42752858, ..., 16.73595127,
         16.96102677, 17.00007724],
        [17.10549598, 16.58310654,         nan, ..., 17.78463126,
         17.77757912, 17.15569575],
        [17.39632988, 17.41838157, 17.57571519, ..., 18.07098448,
         18.36467935, 17.35962105]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.91069974, 16.43307493, 16.42718746, ..., 16.73555945,
         16.96057151, 16.99951095],
        [17.1052305 , 16.58282718,         nan, ..., 17.78415165,
         17.77679681, 17.15536903],
        [17.39597893, 17.41795374, 17.57531644, ..., 18.07053892,
         18.36411567, 17.35921097]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.29702708, ...,  0.        ,
          1.16704883,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.6565437 ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  1.83060692,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -7

In [5]:
ds.SALT

<xarray.DataArray 'SALT' (Z: 52, YC: 60, XC: 240)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76241635, 35.69724222, 35.69800239, ..., 35.66702453,
         35.72830786, 35.78210389],
        [35.80910656, 35.7278929 ,         nan, ..., 35.92798663,
         35.94565591, 35.82460342],
        [35.88534927, 35.83552407, 35.89943901, ..., 35.94890202,
         36.02223367, 35.87853622]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76245369, 35.69726089, 35.69800094, ..., 35.66722655,
         35.72839551, 35.78213623],
        [35.80910656, 35.72788364,         nan, ..., 35.92803851,
         35.94566214, 35.82460342],
        [35.88534164, 35.8354741 , 35.89938586, ..., 35.948915  ,
         36.02224104, 35.87853622]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76246604, 35.69726589, 35.69799823, ..., 35.66740227,
         35.72845882, 35.78214477],
        [35.80910274, 35.72787431,         nan, ..., 35.92806007,
         35.9456524 , 35.82460184],
        [35.8853302 , 35.83543964, 35.89935585, ..., 35.94890432,
         36.02223341, 35.87853241]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        , 12.37136438, ...,  0.        ,
         24.41002681,  0.        ],
        [ 0.        ,  0.        ,         nan, ..., 13.43770223,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ..., 34.86512099,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77

In [6]:
import gsw

def return_rho(pt_values, salt_values, lon_values, lat_values, z_values):
    
    lat_mesh, z_mesh = np.meshgrid(lat_values, z_values)
    pressure_mesh = gsw.p_from_z(z_mesh, lat_mesh)
    pressure_values = np.zeros(np.shape(pt_values))
    lat_grid = np.zeros(np.shape(pt_values))
    lon_grid = np.zeros(np.shape(pt_values))

    for i in range(np.shape(pt_values)[2]):
        pressure_values[:, :, i] = pressure_mesh[:, :]
        lat_grid[:, :, i] = lat_mesh[:, :]
    
    for i in range(np.shape(pt_values)[0]):
        for j in range(np.shape(pt_values)[1]):
            lon_grid[i, j, :] = lon_values[:]
            
    sa_values = gsw.SA_from_SP(salt_values, pressure_values, lon_grid, lat_grid)
    ct_values = gsw.conversions.CT_from_pt(sa_values, pt_values)
    rho_values = gsw.density.rho(sa_values, ct_values, pressure_values)
    print(np.shape(rho_values))
    return rho_values, ct_values, pressure_values
    
rho_values, ct_values, pressure_values = return_rho(ds.where(ds.THETA!=0.0).THETA.values, 
                                                    ds.where(ds.SALT!=0.0).SALT.values, 
                                                    ds.XC.values, ds.YC.values, ds.Z.values)

def rho_values_dataset(dataset, rho_values, propagate=True):
    if propagate:
        attr_d = {}
        for coord in dataset.coords:
            attr_d[coord] = dataset.coords[coord].attrs
    
    da = xr.DataArray(rho_values, dims=["Z", "YC", "XC"], 
                      coords=[ds.Z.values, ds.YC.values, ds.XC.values]).rename('rho')
    
    da.attrs['long_name'] = 'Density'
    da.attrs['units'] = 'kg m^{-3}'
    da.attrs['valid_min'] = 1000 
    da.attrs['valid_max'] = 1100
    
    if propagate:
        for coord in attr_d:
            if coord != 'time':
                da.coords[coord].attrs = attr_d[coord]
    dataset[da.name] = da
    
    if propagate:
        for coord in attr_d:
            dataset.coords[coord].attrs = attr_d[coord]

    return dataset, da


def ct_values_dataset(dataset, ct_values, propagate=True):
    if propagate:
        attr_d = {}
        for coord in dataset.coords:
            attr_d[coord] = dataset.coords[coord].attrs
    
    da = xr.DataArray(ct_values, dims=["Z", "YC", "XC"], 
                      coords=[ds.Z.values, ds.YC.values, ds.XC.values]).rename('ct')
    
    da.attrs['long_name'] = 'Conservative Temperature'
    da.attrs['units'] = 'degC'
    
    if propagate:
        for coord in attr_d:
            if coord != 'time':
                da.coords[coord].attrs = attr_d[coord]
    dataset[da.name] = da
    
    if propagate:
        for coord in attr_d:
            dataset.coords[coord].attrs = attr_d[coord]

    return dataset, da


def pressure_values_dataset(dataset, pressure_values, propagate=True):
    if propagate:
        attr_d = {}
        for coord in dataset.coords:
            attr_d[coord] = dataset.coords[coord].attrs
    
    da = xr.DataArray(pressure_values, dims=["Z", "YC", "XC"], 
                      coords=[ds.Z.values, ds.YC.values, 
                              ds.XC.values]).rename('Pressure')
    
    da.attrs['long_name'] = 'Pressure'
    da.attrs['units'] = 'Pa'
    
    if propagate:
        for coord in attr_d:
            if coord != 'time':
                da.coords[coord].attrs = attr_d[coord]
    dataset[da.name] = da
    
    if propagate:
        for coord in attr_d:
            dataset.coords[coord].attrs = attr_d[coord]

    return dataset, da


ds, da = rho_values_dataset(ds, rho_values)
ds, da = ct_values_dataset(ds, ct_values)
ds, da = pressure_values_dataset(ds, pressure_values)


(52, 60, 240)


In [7]:
np.shape(ds.YC.values)

(60,)

In [8]:
ds.to_netcdf('density_2.nc', engine='netcdf4')

In [9]:
ds.SALT

<xarray.DataArray 'SALT' (Z: 52, YC: 60, XC: 240)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76241635, 35.69724222, 35.69800239, ..., 35.66702453,
         35.72830786, 35.78210389],
        [35.80910656, 35.7278929 ,         nan, ..., 35.92798663,
         35.94565591, 35.82460342],
        [35.88534927, 35.83552407, 35.89943901, ..., 35.94890202,
         36.02223367, 35.87853622]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76245369, 35.69726089, 35.69800094, ..., 35.66722655,
         35.72839551, 35.78213623],
        [35.80910656, 35.72788364,         nan, ..., 35.92803851,
         35.94566214, 35.82460342],
        [35.88534164, 35.8354741 , 35.89938586, ..., 35.948915  ,
         36.02224104, 35.87853622]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [35.76246604, 35.69726589, 35.69799823, ..., 35.66740227,
         35.72845882, 35.78214477],
        [35.80910274, 35.72787431,         nan, ..., 35.92806007,
         35.9456524 , 35.82460184],
        [35.8853302 , 35.83543964, 35.89935585, ..., 35.94890432,
         36.02223341, 35.87853241]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        , 12.37136438, ...,  0.        ,
         24.41002681,  0.        ],
        [ 0.        ,  0.        ,         nan, ..., 13.43770223,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ..., 34.86512099,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,         nan, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]])
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77

In [10]:
ds.ct


<xarray.DataArray 'ct' (Z: 52, YC: 60, XC: 240)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.88146278, 16.40519224, 16.39945762, ..., 16.70858671,
         16.93198019, 16.96971271],
        [17.07379655, 16.55386722,         nan, ..., 17.74972457,
         17.7426305 , 17.12366777],
        [17.36258706, 17.38600847, 17.54158798, ..., 18.0351181 ,
         18.32676366, 17.32617485]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.88047009, 16.40457146, 16.39868918, ..., 16.70785137,
         16.93117363, 16.96871966],
        [17.07334669, 16.55339881,         nan, ..., 17.74882779,
         17.74127163, 17.12309333],
        [17.3619162 , 17.38536914, 17.54082279, ..., 18.03445359,
         18.32586809, 17.32541606]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [16.87992092, 16.40425388, 16.39834853, ..., 16.70745495,
         16.93071694, 16.96815357],
        [17.0730816 , 16.55312001,         nan, ..., 17.74834785,
         17.74049008, 17.12276695],
        [17.36156586, 17.38494262, 17.54042529, ..., 18.03400856,
         18.32530497, 17.3250064 ]],

       ...,

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,  0.3474107 , ...,         nan,
          1.21807258,         nan],
        [        nan,         nan,         nan, ...,  0.71900069,
                 nan,         nan],
        [        nan,         nan,         nan, ...,  1.82907645,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]])
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77.1

AttributeError: 'Dataset' object has no attribute 'pressure'

In [ ]:
da

In [ ]:
def density_pcm_fit_and_post(ds, K=5, maxvar=2):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(0., -max_depth, -10.)
    features_pcm = {'rho': z}
    features = {'rho':'rho'}
    ds = ds.drop(['PCM_LABELS', 'PCM_POST', 'PCM_RANK'])   
 
    attr_d = {}

    for coord in ds.coords:
        attr_d[coord] = ds.coords[coord].attrs
            
    m = pcm(K=K, features=features_pcm, maxvar=maxvar, 
            timeit=True, timeit_verb=1)
    m.fit(ds, features=features, dim='Z') #, inplace=True)
    m.predict(ds, features=features, dim='Z', inplace=True)
    m.predict_proba(ds, features=features, dim='Z', inplace=True)
    m.find_i_metric(ds, inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        del ds.PCM_RANK.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    return ds, m

In [ ]:
ds1, m1 = density_pcm_fit_and_post(ds, K=5, maxvar=2)

In [ ]:
ds1.PCM_LABELS.plot()

In [16]:
ds.THETA.dims

('Z', 'YC', 'XC')

In [17]:
ds.THETA.coords

Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77.16 -76.35 -75.53 ... -31.35 -30.53 -29.72
  * XC       (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9

In [22]:
for coord in ds.THETA.coords:
    print(ds.THETA.coords[coord].values)

2011-08-01T15:12:00.000000000
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
[-77.98265076 -77.16456035 -76.34646994 -75.52837954 -74.71028913
 -73.89219872 -73.07410832 -72.25601791 -71.4379275  -70.6198371
 -69.80174669 -68.98365629 -68.16556588 -67.34747547 -66.52938507
 -65.71129466 -64.89320425 -64.07511385 -63.25702344 -62.43893303
 -61.62084263 -60.80275222 -59.98466181 -59.16657141 -58.348481
 -57.53039059 -56.71230019 -55.89420978 -55.07611937 -54.2580